# What's this PyTorch business?

You've written a lot of code in this assignment to provide a whole host of neural network functionality. Dropout, Batch Norm, and 2D convolutions are some of the workhorses of deep learning in computer vision. You've also worked hard to make your code efficient and vectorized.

For the last part of this assignment, though, we're going to leave behind your beautiful codebase and instead migrate to one of two popular deep learning frameworks: in this instance, PyTorch (or TensorFlow, if you switch over to that notebook). 

### What is PyTorch?

PyTorch is a system for executing dynamic computational graphs over Tensor objects that behave similarly as numpy ndarray. It comes with a powerful automatic differentiation engine that removes the need for manual back-propagation. 

### Why?

* Our code will now run on GPUs! Much faster training. When using a framework like PyTorch or TensorFlow you can harness the power of the GPU for your own custom neural network architectures without having to write CUDA code directly (which is beyond the scope of this class).
* We want you to be ready to use one of these frameworks for your project so you can experiment more efficiently than if you were writing every feature you want to use by hand. 
* We want you to stand on the shoulders of giants! TensorFlow and PyTorch are both excellent frameworks that will make your lives a lot easier, and now that you understand their guts, you are free to use them :) 
* We want you to be exposed to the sort of deep learning code you might run into in academia or industry.

### PyTorch versions
This notebook assumes that you are using **PyTorch version 0.4**. Prior to this version, Tensors had to be wrapped in Variable objects to be used in autograd; however Variables have now been deprecated. In addition 0.4 also separates a Tensor's datatype from its device, and uses numpy-style factories for constructing Tensors rather than directly invoking Tensor constructors.

## How will I learn PyTorch?

Justin Johnson has made an excellent [tutorial](https://github.com/jcjohnson/pytorch-examples) for PyTorch. 

You can also find the detailed [API doc](http://pytorch.org/docs/stable/index.html) here. If you have other questions that are not addressed by the API docs, the [PyTorch forum](https://discuss.pytorch.org/) is a much better place to ask than StackOverflow.


# Table of Contents

This assignment has 5 parts. You will learn PyTorch on different levels of abstractions, which will help you understand it better and prepare you for the final project. 

1. Preparation: we will use CIFAR-10 dataset.
2. Barebones PyTorch: we will work directly with the lowest-level PyTorch Tensors. 
3. PyTorch Module API: we will use `nn.Module` to define arbitrary neural network architecture. 
4. PyTorch Sequential API: we will use `nn.Sequential` to define a linear feed-forward network very conveniently. 
5. CIFAR-10 open-ended challenge: please implement your own network to get as high accuracy as possible on CIFAR-10. You can experiment with any layer, optimizer, hyperparameters or other advanced features. 

Here is a table of comparison:

| API           | Flexibility | Convenience |
|---------------|-------------|-------------|
| Barebone      | High        | Low         |
| `nn.Module`     | High        | Medium      |
| `nn.Sequential` | Low         | High        |

# Part I. Preparation

First, we load the CIFAR-10 dataset. This might take a couple minutes the first time you do it, but the files should stay cached after that.

In previous parts of the assignment we had to write our own code to download the CIFAR-10 dataset, preprocess it, and iterate through it in minibatches; PyTorch provides convenient tools to automate this process for us.

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

In [14]:
NUM_TRAIN = 49000

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.
transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-10
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar10_train = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                             transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, 
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./cs231n/datasets', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./cs231n/datasets', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


You have an option to **use GPU by setting the flag to True below**. It is not necessary to use GPU for this assignment. Note that if your computer does not have CUDA enabled, `torch.cuda.is_available()` will return False and this notebook will fallback to CPU mode.

The global variables `dtype` and `device` will control the data types throughout this assignment. 

In [15]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


# Part II. Barebones PyTorch

PyTorch ships with high-level APIs to help us define model architectures conveniently, which we will cover in Part II of this tutorial. In this section, we will start with the barebone PyTorch elements to understand the autograd engine better. After this exercise, you will come to appreciate the high-level model API more.

We will start with a simple fully-connected ReLU network with two hidden layers and no biases for CIFAR classification. 
This implementation computes the forward pass using operations on PyTorch Tensors, and uses PyTorch autograd to compute gradients. It is important that you understand every line, because you will write a harder version after the example.

When we create a PyTorch Tensor with `requires_grad=True`, then operations involving that Tensor will not just compute values; they will also build up a computational graph in the background, allowing us to easily backpropagate through the graph to compute gradients of some Tensors with respect to a downstream loss. Concretely if x is a Tensor with `x.requires_grad == True` then after backpropagation `x.grad` will be another Tensor holding the gradient of x with respect to the scalar loss at the end.

### PyTorch Tensors: Flatten Function
A PyTorch Tensor is conceptionally similar to a numpy array: it is an n-dimensional grid of numbers, and like numpy PyTorch provides many functions to efficiently operate on Tensors. As a simple example, we provide a `flatten` function below which reshapes image data for use in a fully-connected neural network.

Recall that image data is typically stored in a Tensor of shape N x C x H x W, where:

* N is the number of datapoints
* C is the number of channels
* H is the height of the intermediate feature map in pixels
* W is the height of the intermediate feature map in pixels

This is the right way to represent the data when we are doing something like a 2D convolution, that needs spatial understanding of where the intermediate features are relative to each other. When we use fully connected affine layers to process the image, however, we want each datapoint to be represented by a single vector -- it's no longer useful to segregate the different channels, rows, and columns of the data. So, we use a "flatten" operation to collapse the `C x H x W` values per representation into a single long vector. The flatten function below first reads in the N, C, H, and W values from a given batch of data, and then returns a "view" of that data. "View" is analogous to numpy's "reshape" method: it reshapes x's dimensions to be N x ??, where ?? is allowed to be anything (in this case, it will be C x H x W, but we don't need to specify that explicitly). 

In [16]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

def test_flatten():
    x = torch.arange(12).view(2, 1, 3, 2)
    print('Before flattening: ', x)
    print('After flattening: ', flatten(x))

test_flatten()

Before flattening:  tensor([[[[ 0,  1],
          [ 2,  3],
          [ 4,  5]]],


        [[[ 6,  7],
          [ 8,  9],
          [10, 11]]]])
After flattening:  tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])


### Barebones PyTorch: Two-Layer Network

Here we define a function `two_layer_fc` which performs the forward pass of a two-layer fully-connected ReLU network on a batch of image data. After defining the forward pass we check that it doesn't crash and that it produces outputs of the right shape by running zeros through the network.

You don't have to write any code here, but it's important that you read and understand the implementation.

In [17]:
import torch.nn.functional as F  # useful stateless functions

def two_layer_fc(x, params):
    """
    A fully-connected neural networks; the architecture is:
    NN is fully connected -> ReLU -> fully connected layer.
    Note that this function only defines the forward pass; 
    PyTorch will take care of the backward pass for us.
    
    The input to the network will be a minibatch of data, of shape
    (N, d1, ..., dM) where d1 * ... * dM = D. The hidden layer will have H units,
    and the output layer will produce scores for C classes.
    
    Inputs:
    - x: A PyTorch Tensor of shape (N, d1, ..., dM) giving a minibatch of
      input data.
    - params: A list [w1, w2] of PyTorch Tensors giving weights for the network;
      w1 has shape (D, H) and w2 has shape (H, C).
    
    Returns:
    - scores: A PyTorch Tensor of shape (N, C) giving classification scores for
      the input data x.
    """
    # first we flatten the image
    x = flatten(x)  # shape: [batch_size, C x H x W]
    
    w1, w2 = params
    
    # Forward pass: compute predicted y using operations on Tensors. Since w1 and
    # w2 have requires_grad=True, operations involving these Tensors will cause
    # PyTorch to build a computational graph, allowing automatic computation of
    # gradients. Since we are no longer implementing the backward pass by hand we
    # don't need to keep references to intermediate values.
    # you can also use `.clamp(min=0)`, equivalent to F.relu()
    x = F.relu(x.mm(w1))
    x = x.mm(w2)
    return x
    

def two_layer_fc_test():
    hidden_layer_size = 42
    x = torch.zeros((64, 5, 10), dtype=dtype)  # minibatch size 64, feature dimension 50
    w1 = torch.zeros((50, hidden_layer_size), dtype=dtype)
    w2 = torch.zeros((hidden_layer_size, 10), dtype=dtype)
    scores = two_layer_fc(x, [w1, w2])
    print(scores.size())  # you should see [64, 10]

two_layer_fc_test()

torch.Size([64, 10])


### Barebones PyTorch: Three-Layer ConvNet

Here you will complete the implementation of the function `three_layer_convnet`, which will perform the forward pass of a three-layer convolutional network. Like above, we can immediately test our implementation by passing zeros through the network. The network should have the following architecture:

1. A convolutional layer (with bias) with `channel_1` filters, each with shape `KW1 x KH1`, and zero-padding of two
2. ReLU nonlinearity
3. A convolutional layer (with bias) with `channel_2` filters, each with shape `KW2 x KH2`, and zero-padding of one
4. ReLU nonlinearity
5. Fully-connected layer with bias, producing scores for C classes.

**HINT**: For convolutions: http://pytorch.org/docs/stable/nn.html#torch.nn.functional.conv2d; pay attention to the shapes of convolutional filters!

In [18]:
def three_layer_convnet(x, params):
    """
    Performs the forward pass of a three-layer convolutional network with the
    architecture defined above.

    Inputs:
    - x: A PyTorch Tensor of shape (N, 3, H, W) giving a minibatch of images
    - params: A list of PyTorch Tensors giving the weights and biases for the
      network; should contain the following:
      - conv_w1: PyTorch Tensor of shape (channel_1, 3, KH1, KW1) giving weights
        for the first convolutional layer
      - conv_b1: PyTorch Tensor of shape (channel_1,) giving biases for the first
        convolutional layer
      - conv_w2: PyTorch Tensor of shape (channel_2, channel_1, KH2, KW2) giving
        weights for the second convolutional layer
      - conv_b2: PyTorch Tensor of shape (channel_2,) giving biases for the second
        convolutional layer
      - fc_w: PyTorch Tensor giving weights for the fully-connected layer. Can you
        figure out what the shape should be?
      - fc_b: PyTorch Tensor giving biases for the fully-connected layer. Can you
        figure out what the shape should be?
    
    Returns:
    - scores: PyTorch Tensor of shape (N, C) giving classification scores for x
    """
    conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b = params
    scores = None
    ################################################################################
    # TODO: Implement the forward pass for the three-layer ConvNet.                #
    ################################################################################
    padding1_width = int((conv_w1.shape[2] - 1)/2)
    padding2_width = int((conv_w2.shape[2] - 1)/2)
    x = F.conv2d(x, conv_w1, conv_b1, padding = padding1_width)
    x = F.relu(x)
    x = F.conv2d(x, conv_w2, conv_b2, padding = padding2_width)
    x = F.relu(x)
    x = flatten(x)
    scores = x.mm(fc_w) + fc_b
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################
    return scores

After defining the forward pass of the ConvNet above, run the following cell to test your implementation.

When you run this function, scores should have shape (64, 10).

In [19]:
def three_layer_convnet_test():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]

    conv_w1 = torch.zeros((6, 3, 5, 5), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b1 = torch.zeros((6,))  # out_channel
    print(conv_b1)
    conv_w2 = torch.zeros((9, 6, 3, 3), dtype=dtype)  # [out_channel, in_channel, kernel_H, kernel_W]
    conv_b2 = torch.zeros((9,))  # out_channel

    # you must calculate the shape of the tensor after two conv layers, before the fully-connected layer
    fc_w = torch.zeros((9 * 32 * 32, 10))
    fc_b = torch.zeros(10)
    print (fc_b)

    scores = three_layer_convnet(x, [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b])
    print(scores.size())  # you should see [64, 10]
three_layer_convnet_test()

tensor([0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
torch.Size([64, 10])


### Barebones PyTorch: Initialization
Let's write a couple utility methods to initialize the weight matrices for our models.

- `random_weight(shape)` initializes a weight tensor with the Kaiming normalization method.
- `zero_weight(shape)` initializes a weight tensor with all zeros. Useful for instantiating bias parameters.

The `random_weight` function uses the Kaiming normal initialization method, described in:

He et al, [*Delving Deep into Rectifiers: Surpassing Human-Level Performance on ImageNet Classification*](https://arxiv.org/abs/1502.01852), ICCV 2015, 

In [20]:
def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator. 
    w = torch.randn(shape, device=device, dtype=dtype)* np.sqrt(2. / fan_in) 
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU. 
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))
#zero_weight((3, 5, 3, 2))

tensor([[ 0.1313, -1.7417,  0.6270, -0.5272, -0.8837],
        [-1.2573,  1.6363, -0.9550,  0.0906, -1.3353],
        [ 0.0610, -1.0217, -1.1637, -0.2684, -0.1314]],
       device='cuda:0', requires_grad=True)

### Barebones PyTorch: Check Accuracy
When training the model we will use the following function to check the accuracy of our model on the training or validation sets.

When checking accuracy we don't need to compute any gradients; as a result we don't need PyTorch to build a computational graph for us when we compute scores. To prevent a graph from being built we scope our computation under a `torch.no_grad()` context manager.

In [21]:
def check_accuracy_part2(loader, model_fn, params):
    """
    Check the accuracy of a classification model.
    
    Inputs:
    - loader: A DataLoader for the data split we want to check
    - model_fn: A function that performs the forward pass of the model,
      with the signature scores = model_fn(x, params)
    - params: List of PyTorch Tensors giving parameters of the model
    
    Returns: Nothing, but prints the accuracy of the model
    """
    split = 'val' if loader.dataset.train else 'test'
    print('Checking accuracy on the %s set' % split)
    num_correct, num_samples = 0, 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.int64)
            scores = model_fn(x, params)
            print(scores.max(1))
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f%%)' % (num_correct, num_samples, 100 * acc))

### BareBones PyTorch: Training Loop
We can now set up a basic training loop to train our network. We will train the model using stochastic gradient descent without momentum. We will use `torch.functional.cross_entropy` to compute the loss; you can [read about it here](http://pytorch.org/docs/stable/nn.html#cross-entropy).

The training loop takes as input the neural network function, a list of initialized parameters (`[w1, w2]` in our example), and learning rate.

In [22]:
def train_part2(model_fn, params, learning_rate):
    """
    Train a model on CIFAR-10.
    
    Inputs:
    - model_fn: A Python function that performs the forward pass of the model.
      It should have the signature scores = model_fn(x, params) where x is a
      PyTorch Tensor of image data, params is a list of PyTorch Tensors giving
      model weights, and scores is a PyTorch Tensor of shape (N, C) giving
      scores for the elements in x.
    - params: List of PyTorch Tensors giving weights for the model
    - learning_rate: Python scalar giving the learning rate to use for SGD
    
    Returns: Nothing
    """
    for t, (x, y) in enumerate(loader_train):
        # Move the data to the proper device (GPU or CPU)
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)

        # Forward pass: compute scores and loss
        scores = model_fn(x, params)
        loss = F.cross_entropy(scores, y)

        # Backward pass: PyTorch figures out which Tensors in the computational
        # graph has requires_grad=True and uses backpropagation to compute the
        # gradient of the loss with respect to these Tensors, and stores the
        # gradients in the .grad attribute of each Tensor.
        loss.backward()

        # Update parameters. We don't want to backpropagate through the
        # parameter updates, so we scope the updates under a torch.no_grad()
        # context manager to prevent a computational graph from being built.
        with torch.no_grad():
            for w in params:
                w -= learning_rate * w.grad

                # Manually zero the gradients after running the backward pass
                w.grad.zero_()

        if t % print_every == 0:
            print('Iteration %d, loss = %.4f' % (t, loss.item()))
            check_accuracy_part2(loader_val, model_fn, params)
            print()

### BareBones PyTorch: Train a Two-Layer Network
Now we are ready to run the training loop. We need to explicitly allocate tensors for the fully connected weights, `w1` and `w2`. 

Each minibatch of CIFAR has 64 examples, so the tensor shape is `[64, 3, 32, 32]`. 

After flattening, `x` shape should be `[64, 3 * 32 * 32]`. This will be the size of the first dimension of `w1`. 
The second dimension of `w1` is the hidden layer size, which will also be the first dimension of `w2`. 

Finally, the output of the network is a 10-dimensional vector that represents the probability distribution over 10 classes. 

You don't need to tune any hyperparameters but you should see accuracies above 40% after training for one epoch.

In [23]:
hidden_layer_size = 4000
learning_rate = 1e-2

w1 = random_weight((3 * 32 * 32, hidden_layer_size))
w2 = random_weight((hidden_layer_size, 10))

train_part2(two_layer_fc, [w1, w2], learning_rate)

Iteration 0, loss = 3.2607
Checking accuracy on the val set
(tensor([0.2110, 2.6757, 2.1840, 4.3568, 1.9447, 1.2247, 2.3386, 2.2822, 2.1727,
        3.4136, 2.1211, 1.8365, 2.7088, 1.9390, 2.8517, 3.5560, 1.3196, 2.6604,
        1.7495, 0.8396, 2.7794, 1.3854, 3.3914, 3.5120, 1.4729, 1.2034, 3.4601,
        4.3516, 4.0807, 1.6552, 2.1873, 0.9909, 5.8207, 2.3618, 2.2812, 2.9554,
        2.1885, 1.2349, 2.1995, 0.8133, 0.6336, 1.2072, 1.0063, 2.3028, 3.1074,
        2.2745, 1.4952, 5.8089, 2.0236, 3.0927, 1.3696, 4.3784, 1.0572, 3.7092,
        1.4539, 1.1682, 2.2885, 4.0787, 2.2876, 2.1472, 2.1981, 3.0129, 2.3028,
        1.7054], device='cuda:0'), tensor([7, 3, 7, 1, 1, 1, 7, 3, 3, 7, 7, 5, 5, 9, 3, 3, 1, 5, 3, 8, 7, 3, 3, 1,
        1, 1, 1, 1, 7, 3, 0, 5, 5, 1, 1, 4, 5, 7, 3, 1, 7, 7, 7, 3, 7, 3, 3, 3,
        7, 1, 3, 8, 3, 3, 3, 1, 3, 1, 3, 7, 2, 2, 1, 1], device='cuda:0'))
(tensor([ 1.8164,  2.2746,  2.3827,  4.9373,  3.9457,  2.6681,  4.2991,  4.6791,
         1.6297,  1.9836,  1

(tensor([2.3622, 2.7514, 2.8851, 3.5884, 3.8001, 1.0904, 3.1995, 1.9226, 0.9765,
        1.9744, 1.4642, 1.8940, 2.7976, 2.7246, 3.8649, 1.9107, 1.4446, 0.8403,
        3.2605, 3.4962, 2.4956, 1.9312, 3.5404, 1.1771, 1.2909, 1.9381, 2.0934,
        2.5227, 1.9752, 3.9706, 0.7384, 2.1532, 2.7376, 3.1935, 2.7894, 2.7773,
        2.7203, 2.6468, 4.2986, 2.4495, 1.0354, 1.1650, 1.8479, 3.7468, 3.4932,
        2.6207, 2.8280, 0.3856, 1.8231, 2.8206, 2.9451, 6.0192, 3.8856, 1.0202,
        2.9069, 2.6416, 1.1921, 1.3692, 3.1422, 2.8136, 2.9523, 2.5505, 1.0627,
        2.4028], device='cuda:0'), tensor([3, 1, 2, 3, 3, 5, 1, 3, 5, 3, 8, 7, 3, 5, 5, 2, 3, 1, 1, 5, 1, 3, 3, 3,
        1, 3, 0, 1, 1, 1, 9, 1, 1, 1, 7, 1, 7, 1, 1, 1, 1, 3, 3, 1, 3, 1, 1, 7,
        7, 1, 1, 1, 3, 3, 7, 1, 1, 3, 5, 3, 7, 8, 7, 3], device='cuda:0'))
(tensor([1.1012, 3.6380, 2.7838, 1.6950, 3.7119, 2.9444, 1.7843, 2.1711, 2.8287,
        2.9087, 2.8649, 2.0540, 1.6948, 3.4544, 1.7933, 2.7933, 4.1743, 2.7320,
        

Iteration 200, loss = 2.0492
Checking accuracy on the val set
(tensor([4.6639, 2.4722, 2.5701, 2.2922, 2.4839, 1.1386, 1.9069, 2.3096, 1.7123,
        3.6139, 3.1799, 3.0961, 5.9857, 4.2345, 1.7591, 3.0056, 3.6896, 3.0476,
        3.3736, 5.6846, 1.3518, 3.7788, 3.1396, 4.7685, 4.1744, 7.3411, 3.8374,
        3.6040, 2.4746, 1.2802, 4.9306, 2.4661, 1.3026, 1.7719, 5.9054, 2.8740,
        2.1942, 7.8396, 1.1135, 1.6525, 2.6232, 4.3779, 3.7803, 3.6453, 3.5484,
        2.9170, 0.7887, 4.9814, 2.5298, 0.7939, 5.2320, 1.5876, 4.8797, 3.8605,
        7.0316, 3.1619, 2.6070, 3.1364, 1.7871, 8.0616, 3.2201, 3.3122, 3.9355,
        4.7970], device='cuda:0'), tensor([0, 1, 2, 6, 4, 0, 6, 0, 5, 6, 5, 5, 0, 8, 1, 0, 1, 6, 7, 1, 7, 6, 7, 6,
        7, 0, 6, 0, 7, 9, 7, 5, 2, 0, 0, 5, 5, 0, 8, 6, 4, 6, 8, 1, 7, 8, 2, 6,
        0, 8, 8, 5, 7, 1, 0, 6, 6, 1, 8, 0, 8, 4, 0, 9], device='cuda:0'))
(tensor([1.2526, 1.4911, 3.0254, 4.4895, 5.1254, 2.2046, 3.5200, 2.0883, 8.1267,
        0.3948, 5.0967, 4.

Iteration 300, loss = 1.7219
Checking accuracy on the val set
(tensor([2.4235, 5.0043, 2.5491, 0.8524, 3.0020, 4.6565, 6.9899, 1.7910, 0.1701,
        1.5550, 3.9774, 6.0132, 3.2266, 3.8005, 2.1926, 2.2910, 2.6002, 4.4109,
        4.9896, 5.5462, 2.3541, 0.9765, 2.0993, 2.1426, 2.0539, 1.1189, 3.8034,
        3.4388, 3.7317, 3.8544, 4.4085, 1.3742, 3.8402, 2.2087, 3.5438, 1.8665,
        3.3281, 0.8941, 3.3947, 2.5126, 3.6559, 3.1030, 2.2635, 4.4299, 2.7218,
        1.7521, 1.5955, 1.9508, 2.4921, 3.8983, 2.2641, 1.9524, 1.4532, 3.5996,
        2.2197, 1.6696, 5.5223, 5.1187, 3.5931, 3.9810, 3.1497, 3.8004, 1.5327,
        2.2578], device='cuda:0'), tensor([7, 0, 7, 8, 3, 8, 6, 5, 6, 5, 3, 8, 2, 2, 2, 5, 1, 1, 5, 3, 2, 2, 7, 8,
        2, 5, 8, 7, 5, 3, 5, 3, 3, 8, 9, 5, 9, 8, 5, 5, 5, 7, 5, 8, 5, 2, 8, 7,
        5, 9, 8, 8, 7, 8, 3, 8, 8, 9, 2, 5, 8, 4, 5, 5], device='cuda:0'))
(tensor([1.3016, 5.7881, 4.5011, 1.4362, 4.7142, 4.8141, 3.1338, 1.4133, 2.1694,
        3.1205, 2.2814, 1.

Iteration 400, loss = 1.8531
Checking accuracy on the val set
(tensor([3.6853, 2.0421, 2.6637, 1.8775, 2.7304, 2.7052, 3.2522, 2.8316, 5.0668,
        2.4012, 0.8901, 3.7177, 2.2972, 1.4347, 3.7739, 3.4381, 2.4245, 2.6991,
        2.6690, 4.0090, 3.4951, 2.3517, 1.4189, 4.9351, 2.5738, 4.2564, 2.9736,
        2.4273, 1.3989, 2.3049, 2.0868, 4.4010, 2.8896, 4.2678, 2.5962, 1.7803,
        1.1262, 4.3092, 1.1053, 1.4123, 8.2586, 2.6805, 3.2314, 4.6931, 1.6176,
        1.1628, 0.8691, 3.5542, 1.9686, 1.6483, 1.4706, 6.5546, 2.5818, 3.7118,
        2.7706, 1.6986, 5.7240, 1.9699, 2.2892, 2.3941, 4.3987, 0.8335, 4.4398,
        2.5126], device='cuda:0'), tensor([0, 0, 1, 6, 3, 8, 6, 2, 9, 2, 2, 9, 6, 6, 0, 9, 8, 9, 6, 5, 9, 0, 1, 2,
        5, 8, 6, 2, 6, 9, 2, 9, 6, 0, 2, 0, 8, 2, 2, 3, 0, 9, 7, 0, 2, 2, 9, 1,
        2, 2, 2, 5, 4, 9, 6, 5, 9, 3, 1, 9, 8, 6, 0, 9], device='cuda:0'))
(tensor([2.5161, 1.5649, 2.0382, 1.6916, 4.3710, 2.1765, 2.1600, 5.1042, 0.9559,
        2.3132, 2.5542, 2.

Iteration 500, loss = 1.6887
Checking accuracy on the val set
(tensor([1.9031, 3.4826, 3.4171, 4.4537, 4.0043, 3.0708, 4.0871, 1.4716, 3.4670,
        1.8741, 5.8590, 2.1716, 2.9351, 4.2829, 4.1911, 3.3815, 2.1712, 6.0631,
        2.4169, 6.4459, 2.2100, 1.5875, 4.1187, 2.9472, 3.4396, 4.1344, 1.7638,
        3.6874, 6.5014, 4.2300, 1.9957, 6.8395, 4.2626, 3.6548, 3.7486, 1.6296,
        2.5826, 1.1081, 3.1976, 6.1596, 2.1632, 4.0056, 2.7912, 3.3252, 0.9246,
        2.6491, 4.9686, 2.9460, 4.9268, 1.5635, 5.0523, 5.9880, 6.8269, 3.3473,
        2.7127, 5.7484, 3.7130, 1.4712, 0.7402, 4.1788, 5.0312, 2.9653, 2.7787,
        1.3143], device='cuda:0'), tensor([5, 8, 8, 7, 1, 5, 8, 1, 8, 6, 8, 8, 9, 8, 0, 4, 5, 8, 4, 8, 5, 5, 5, 8,
        6, 8, 8, 6, 8, 8, 1, 7, 6, 9, 8, 6, 5, 2, 7, 0, 5, 6, 6, 6, 7, 5, 6, 4,
        4, 8, 5, 8, 5, 9, 8, 9, 5, 8, 9, 1, 1, 5, 5, 1], device='cuda:0'))
(tensor([1.4439, 6.2762, 3.6274, 3.2067, 2.9637, 5.8501, 3.2855, 3.4327, 2.1573,
        4.8532, 5.7996, 3.

Iteration 600, loss = 1.8774
Checking accuracy on the val set
(tensor([ 3.8971,  3.0814,  3.0895,  3.0476,  2.3391,  3.5114, -0.0698,  7.3859,
         2.1185,  5.4889,  3.7916,  1.3024,  5.5520,  2.4534,  0.8938,  2.3145,
         2.5845,  2.3404,  3.0691,  5.2030,  2.1851,  2.8010,  0.9159,  1.2322,
         4.2750,  3.9945,  6.0982,  5.1902,  1.3788,  1.3742,  3.0424,  1.8603,
         1.1659,  1.7541,  1.3377,  5.4816,  4.5451,  4.8548,  3.5099,  6.6787,
         2.9020,  3.2817,  2.9166,  2.7122,  1.5859,  2.7375,  1.9413,  3.9717,
         1.3856,  2.4833,  4.8716,  4.9322,  3.6515,  3.9952,  2.2932,  2.8259,
         5.5377,  5.1564,  8.5628,  3.2741,  3.1905,  4.0633,  2.8622,  3.1330],
       device='cuda:0'), tensor([7, 6, 0, 1, 9, 7, 3, 0, 0, 8, 6, 3, 8, 8, 7, 2, 2, 3, 0, 3, 2, 7, 6, 3,
        0, 2, 3, 0, 1, 0, 0, 0, 0, 3, 0, 8, 8, 8, 9, 3, 3, 9, 9, 0, 3, 0, 0, 7,
        4, 0, 8, 1, 0, 8, 9, 6, 8, 6, 0, 8, 0, 7, 8, 2], device='cuda:0'))
(tensor([2.2391, 2.7208, 4.0345, 1.5

Iteration 700, loss = 1.5794
Checking accuracy on the val set
(tensor([2.2453, 3.9665, 3.6151, 2.5105, 4.6456, 4.2612, 4.4305, 2.1636, 1.8149,
        5.5603, 2.2333, 0.9524, 3.2817, 3.7167, 3.4659, 4.5546, 1.2186, 1.1888,
        2.9269, 2.3142, 1.1824, 1.7580, 5.8152, 2.2180, 5.1075, 2.5368, 3.7846,
        2.6150, 1.8090, 1.3499, 2.1736, 1.0376, 2.9677, 1.3541, 1.9271, 3.2528,
        1.5001, 0.4597, 1.8562, 2.0946, 2.2846, 4.2692, 1.5629, 3.8936, 2.0423,
        3.9528, 2.0275, 3.2773, 2.7981, 3.3932, 4.2946, 1.9201, 1.0807, 2.3443,
        2.4832, 6.8362, 4.6384, 2.5249, 1.1438, 3.2359, 3.2234, 3.1454, 2.9617,
        2.2783], device='cuda:0'), tensor([9, 3, 1, 6, 1, 8, 3, 6, 9, 9, 6, 5, 7, 8, 8, 6, 1, 4, 3, 2, 9, 9, 1, 5,
        6, 6, 1, 2, 5, 8, 4, 6, 5, 1, 2, 5, 8, 2, 6, 5, 2, 6, 1, 8, 1, 2, 8, 1,
        1, 5, 1, 9, 1, 5, 1, 1, 3, 1, 1, 6, 1, 1, 5, 1], device='cuda:0'))
(tensor([7.3304, 1.9703, 4.4048, 2.6155, 4.2089, 0.7786, 0.8593, 3.0103, 2.4231,
        1.8525, 4.9662, 4.

### BareBones PyTorch: Training a ConvNet

In the below you should use the functions defined above to train a three-layer convolutional network on CIFAR. The network should have the following architecture:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You don't need to tune any hyperparameters, but if everything works correctly you should achieve an accuracy above 42% after one epoch.

In [12]:
learning_rate = 3e-3

channel_1 = 32
channel_2 = 16

conv_w1 = None
conv_b1 = None
conv_w2 = None
conv_b2 = None
fc_w = None
fc_b = None

################################################################################
# TODO: Initialize the parameters of a three-layer ConvNet.                    #
################################################################################
conv_w1 = random_weight((9, 3, 5, 5))
conv_b1 = zero_weight((9,))
conv_w2 = random_weight((6, 9, 3, 3))
conv_b2 = zero_weight((6,))
fc_w = random_weight((6 * 32 * 32, 10 ))
fc_b = zero_weight((10 ))
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

params = [conv_w1, conv_b1, conv_w2, conv_b2, fc_w, fc_b]
train_part2(three_layer_convnet, params, learning_rate)

Iteration 0, loss = 2.8583
Checking accuracy on the val set
Got 91 / 1000 correct (9.10%)

Iteration 100, loss = 1.9955
Checking accuracy on the val set
Got 268 / 1000 correct (26.80%)

Iteration 200, loss = 1.9013
Checking accuracy on the val set
Got 319 / 1000 correct (31.90%)

Iteration 300, loss = 1.6928
Checking accuracy on the val set
Got 353 / 1000 correct (35.30%)

Iteration 400, loss = 1.7549
Checking accuracy on the val set
Got 383 / 1000 correct (38.30%)

Iteration 500, loss = 1.6830
Checking accuracy on the val set
Got 403 / 1000 correct (40.30%)

Iteration 600, loss = 1.7110
Checking accuracy on the val set
Got 399 / 1000 correct (39.90%)

Iteration 700, loss = 1.6193
Checking accuracy on the val set
Got 425 / 1000 correct (42.50%)



# Part III. PyTorch Module API

Barebone PyTorch requires that we track all the parameter tensors by hand. This is fine for small networks with a few tensors, but it would be extremely inconvenient and error-prone to track tens or hundreds of tensors in larger networks.

PyTorch provides the `nn.Module` API for you to define arbitrary network architectures, while tracking every learnable parameters for you. In Part II, we implemented SGD ourselves. PyTorch also provides the `torch.optim` package that implements all the common optimizers, such as RMSProp, Adagrad, and Adam. It even supports approximate second-order methods like L-BFGS! You can refer to the [doc](http://pytorch.org/docs/master/optim.html) for the exact specifications of each optimizer.

To use the Module API, follow the steps below:

1. Subclass `nn.Module`. Give your network class an intuitive name like `TwoLayerFC`. 

2. In the constructor `__init__()`, define all the layers you need as class attributes. Layer objects like `nn.Linear` and `nn.Conv2d` are themselves `nn.Module` subclasses and contain learnable parameters, so that you don't have to instantiate the raw tensors yourself. `nn.Module` will track these internal parameters for you. Refer to the [doc](http://pytorch.org/docs/master/nn.html) to learn more about the dozens of builtin layers. **Warning**: don't forget to call the `super().__init__()` first!

3. In the `forward()` method, define the *connectivity* of your network. You should use the attributes defined in `__init__` as function calls that take tensor as input and output the "transformed" tensor. Do *not* create any new layers with learnable parameters in `forward()`! All of them must be declared upfront in `__init__`. 

After you define your Module subclass, you can instantiate it as an object and call it just like the NN forward function in part II.

### Module API: Two-Layer Network
Here is a concrete example of a 2-layer fully connected network:

In [13]:
class TwoLayerFC(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        # assign layer objects to class attributes
        self.fc1 = nn.Linear(input_size, hidden_size)
        # nn.init package contains convenient initialization methods
        # http://pytorch.org/docs/master/nn.html#torch-nn-init 
        nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        nn.init.kaiming_normal_(self.fc2.weight)
    
    def forward(self, x):
        # forward always defines connectivity
        x = flatten(x)
        scores = self.fc2(F.relu(self.fc1(x)))
        return scores

def test_TwoLayerFC():
    input_size = 50
    x = torch.zeros((64, input_size), dtype=dtype)  # minibatch size 64, feature dimension 50
    model = TwoLayerFC(input_size, 42, 10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_TwoLayerFC()

torch.Size([64, 10])


### Module API: Three-Layer ConvNet
It's your turn to implement a 3-layer ConvNet followed by a fully connected layer. The network architecture should be the same as in Part II:

1. Convolutional layer with `channel_1` 5x5 filters with zero-padding of 2
2. ReLU
3. Convolutional layer with `channel_2` 3x3 filters with zero-padding of 1
4. ReLU
5. Fully-connected layer to `num_classes` classes

You should initialize the weight matrices of the model using the Kaiming normal initialization method.

**HINT**: http://pytorch.org/docs/stable/nn.html#conv2d

After you implement the three-layer ConvNet, the `test_ThreeLayerConvNet` function will run your implementation; it should print `(64, 10)` for the shape of the output scores.

In [14]:
class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        ########################################################################
        # TODO: Set up the layers you need for a three-layer ConvNet with the  #
        # architecture defined above.     
        # in_channel: color dim * length * width for image
        # channel_1: out_channel, basically channel_1 * 5 * 5 filters, 
        # 5: filter size if it is square
        # fc: channel_2 * 32 * 32 -- 32 * 32 the size of the adjusted image len&width
        ########################################################################
        self.cn1 = nn.Conv2d(in_channel, channel_1, 5, padding = 2)
        nn.init.kaiming_normal_(self.cn1.weight)
        self.cn2 = nn.Conv2d(channel_1, channel_2, 3, padding = 1 )
        nn.init.kaiming_normal_(self.cn2.weight)
        self.fc = nn.Linear(channel_2 * 32 * 32, num_classes )
        nn.init.kaiming_normal_(self.fc.weight)
        
        ########################################################################
        #                          END OF YOUR CODE                            #       
        ########################################################################

    def forward(self, x):
        scores = None
        ########################################################################
        # TODO: Implement the forward function for a 3-layer ConvNet. you      #
        # should use the layers you defined in __init__ and specify the        #
        # connectivity of those layers in forward()                            #
        ########################################################################
        x = F.relu(self.cn2(F.relu(self.cn1(x))))
        x = flatten(x)
        scores = self.fc(x)
        ########################################################################
        #                             END OF YOUR CODE                         #
        ########################################################################
        return scores


def test_ThreeLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)  # minibatch size 64, image size [3, 32, 32]
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.size())  # you should see [64, 10]
test_ThreeLayerConvNet()

torch.Size([64, 10])


### Module API: Check Accuracy
Given the validation or test set, we can check the classification accuracy of a neural network. 

This version is slightly different from the one in part II. You don't manually pass in the parameters anymore.

In [15]:
def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

### Module API: Training Loop
We also use a slightly different training loop. Rather than updating the values of the weights ourselves, we use an Optimizer object from the `torch.optim` package, which abstract the notion of an optimization algorithm and provides implementations of most of the algorithms commonly used to optimize neural networks.

In [16]:
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                acc = check_accuracy_part34(loader_val, model)
                print()
    return acc

### Module API: Train a Two-Layer Network
Now we are ready to run the training loop. In contrast to part II, we don't explicitly allocate parameter tensors anymore.

Simply pass the input size, hidden layer size, and number of classes (i.e. output size) to the constructor of `TwoLayerFC`. 

You also need to define an optimizer that tracks all the learnable parameters inside `TwoLayerFC`.

You don't need to tune any hyperparameters, but you should see model accuracies above 40% after training for one epoch.

In [17]:
hidden_layer_size = 4000
learning_rate = 1e-2
model = TwoLayerFC(3 * 32 * 32, hidden_layer_size, 10)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

train_part34(model, optimizer)

Iteration 0, loss = 4.4503
Checking accuracy on validation set
Got 146 / 1000 correct (14.60)

Iteration 100, loss = 2.3197
Checking accuracy on validation set
Got 336 / 1000 correct (33.60)

Iteration 200, loss = 2.1945
Checking accuracy on validation set
Got 385 / 1000 correct (38.50)

Iteration 300, loss = 2.3833
Checking accuracy on validation set
Got 350 / 1000 correct (35.00)

Iteration 400, loss = 1.9778
Checking accuracy on validation set
Got 390 / 1000 correct (39.00)

Iteration 500, loss = 1.8271
Checking accuracy on validation set
Got 415 / 1000 correct (41.50)

Iteration 600, loss = 2.0897
Checking accuracy on validation set
Got 403 / 1000 correct (40.30)

Iteration 700, loss = 1.6785
Checking accuracy on validation set
Got 439 / 1000 correct (43.90)



### Module API: Train a Three-Layer ConvNet
You should now use the Module API to train a three-layer ConvNet on CIFAR. This should look very similar to training the two-layer network! You don't need to tune any hyperparameters, but you should achieve above above 45% after training for one epoch.

You should train the model using stochastic gradient descent without momentum.

In [18]:
learning_rate = 3e-3
channel_1 = 32
channel_2 = 16

model = None
optimizer = None
################################################################################
# TODO: Instantiate your ThreeLayerConvNet model and a corresponding optimizer #
################################################################################
in_channel = 3
num_classes = 10
model = ThreeLayerConvNet(in_channel, channel_1, channel_2, num_classes)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 2.5181
Checking accuracy on validation set
Got 126 / 1000 correct (12.60)

Iteration 100, loss = 2.0136
Checking accuracy on validation set
Got 370 / 1000 correct (37.00)

Iteration 200, loss = 1.8367
Checking accuracy on validation set
Got 400 / 1000 correct (40.00)

Iteration 300, loss = 1.6443
Checking accuracy on validation set
Got 402 / 1000 correct (40.20)

Iteration 400, loss = 1.6314
Checking accuracy on validation set
Got 444 / 1000 correct (44.40)

Iteration 500, loss = 1.4143
Checking accuracy on validation set
Got 456 / 1000 correct (45.60)

Iteration 600, loss = 1.6374
Checking accuracy on validation set
Got 465 / 1000 correct (46.50)

Iteration 700, loss = 1.5459
Checking accuracy on validation set
Got 471 / 1000 correct (47.10)



# Part IV. PyTorch Sequential API

Part III introduced the PyTorch Module API, which allows you to define arbitrary learnable layers and their connectivity. 

For simple models like a stack of feed forward layers, you still need to go through 3 steps: subclass `nn.Module`, assign layers to class attributes in `__init__`, and call each layer one by one in `forward()`. Is there a more convenient way? 

Fortunately, PyTorch provides a container Module called `nn.Sequential`, which merges the above steps into one. It is not as flexible as `nn.Module`, because you cannot specify more complex topology than a feed-forward stack, but it's good enough for many use cases.

### Sequential API: Two-Layer Network
Let's see how to rewrite our two-layer fully connected network example with `nn.Sequential`, and train it using the training loop defined above.

Again, you don't need to tune any hyperparameters here, but you shoud achieve above 40% accuracy after one epoch of training.

In [19]:
# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

hidden_layer_size = 4000
learning_rate = 1e-2

model = nn.Sequential(
    Flatten(),
    nn.Linear(3 * 32 * 32, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, 10),
)

# you can use Nesterov momentum in optim.SGD
optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

train_part34(model, optimizer)

Iteration 0, loss = 2.3739
Checking accuracy on validation set
Got 165 / 1000 correct (16.50)

Iteration 100, loss = 1.6380
Checking accuracy on validation set
Got 346 / 1000 correct (34.60)

Iteration 200, loss = 1.7930
Checking accuracy on validation set
Got 403 / 1000 correct (40.30)

Iteration 300, loss = 1.7927
Checking accuracy on validation set
Got 425 / 1000 correct (42.50)

Iteration 400, loss = 2.1293
Checking accuracy on validation set
Got 446 / 1000 correct (44.60)

Iteration 500, loss = 1.8028
Checking accuracy on validation set
Got 424 / 1000 correct (42.40)

Iteration 600, loss = 1.8161
Checking accuracy on validation set
Got 422 / 1000 correct (42.20)

Iteration 700, loss = 1.3567
Checking accuracy on validation set
Got 439 / 1000 correct (43.90)



### Sequential API: Three-Layer ConvNet
Here you should use `nn.Sequential` to define and train a three-layer ConvNet with the same architecture we used in Part III:

1. Convolutional layer (with bias) with 32 5x5 filters, with zero-padding of 2
2. ReLU
3. Convolutional layer (with bias) with 16 3x3 filters, with zero-padding of 1
4. ReLU
5. Fully-connected layer (with bias) to compute scores for 10 classes

You should initialize your weight matrices using the `random_weight` function defined above, and you should initialize your bias vectors using the `zero_weight` function above.

You should optimize your model using stochastic gradient descent with Nesterov momentum 0.9.

Again, you don't need to tune any hyperparameters but you should see accuracy above 55% after one epoch of training.

In [20]:
channel_1 = 32
channel_2 = 16
learning_rate = 1e-2

model = None
optimizer = None

################################################################################
# TODO: Rewrite the 2-layer ConvNet with bias from Part III with the           #
# Sequential API.                                                              #
################################################################################
in_channel = 3
num_classes = 10
model = nn.Sequential(
    nn.Conv2d(in_channel, channel_1, 5, padding = 2),
    nn.ReLU(),
    nn.Conv2d(channel_1, channel_2, 3, padding = 1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(channel_2 * 32 * 32, num_classes)    
    )

optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)
################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer)

Iteration 0, loss = 2.3049
Checking accuracy on validation set
Got 113 / 1000 correct (11.30)

Iteration 100, loss = 1.6744
Checking accuracy on validation set
Got 464 / 1000 correct (46.40)

Iteration 200, loss = 1.6096
Checking accuracy on validation set
Got 481 / 1000 correct (48.10)

Iteration 300, loss = 1.4730
Checking accuracy on validation set
Got 517 / 1000 correct (51.70)

Iteration 400, loss = 1.1886
Checking accuracy on validation set
Got 532 / 1000 correct (53.20)

Iteration 500, loss = 1.3347
Checking accuracy on validation set
Got 554 / 1000 correct (55.40)

Iteration 600, loss = 1.1639
Checking accuracy on validation set
Got 581 / 1000 correct (58.10)

Iteration 700, loss = 0.9543
Checking accuracy on validation set
Got 588 / 1000 correct (58.80)



# Part V. CIFAR-10 open-ended challenge

In this section, you can experiment with whatever ConvNet architecture you'd like on CIFAR-10. 

Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves **at least 70%** accuracy on the CIFAR-10 **validation** set within 10 epochs. You can use the check_accuracy and train functions from above. You can use either `nn.Module` or `nn.Sequential` API. 

Describe what you did at the end of this notebook.

Here are the official API documentation for each component. One note: what we call in the class "spatial batch norm" is called "BatchNorm2D" in PyTorch.

* Layers in torch.nn package: http://pytorch.org/docs/stable/nn.html
* Activations: http://pytorch.org/docs/stable/nn.html#non-linear-activations
* Loss functions: http://pytorch.org/docs/stable/nn.html#loss-functions
* Optimizers: http://pytorch.org/docs/stable/optim.html


### Things you might try:
- **Filter size**: Above we used 5x5; would smaller filters be more efficient?
- **Number of filters**: Above we used 32 filters. Do more or fewer do better?
- **Pooling vs Strided Convolution**: Do you use max pooling or just stride convolutions?
- **Batch normalization**: Try adding spatial batch normalization after convolution layers and vanilla batch normalization after affine layers. Do your networks train faster?
- **Network architecture**: The network above has two layers of trainable parameters. Can you do better with a deep network? Good architectures to try include:
    - [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [conv-relu-conv-relu-pool]xN -> [affine]xM -> [softmax or SVM]
    - [batchnorm-relu-conv]xN -> [affine]xM -> [softmax or SVM]
- **Global Average Pooling**: Instead of flattening and then having multiple affine layers, perform convolutions until your image gets small (7x7 or so) and then perform an average pooling operation to get to a 1x1 image picture (1, 1 , Filter#), which is then reshaped into a (Filter#) vector. This is used in [Google's Inception Network](https://arxiv.org/abs/1512.00567) (See Table 1 for their architecture).
- **Regularization**: Add l2 weight regularization, or perhaps use Dropout.

### Tips for training
For each network architecture that you try, you should tune the learning rate and other hyperparameters. When doing this there are a couple important things to keep in mind:

- If the parameters are working well, you should see improvement within a few hundred iterations
- Remember the coarse-to-fine approach for hyperparameter tuning: start by testing a large range of hyperparameters for just a few training iterations to find the combinations of parameters that are working at all.
- Once you have found some sets of parameters that seem to work, search more finely around these parameters. You may need to train for more epochs.
- You should use the validation set for hyperparameter search, and save your test set for evaluating your architecture on the best parameters as selected by the validation set.

### Going above and beyond
If you are feeling adventurous there are many other features you can implement to try and improve your performance. You are **not required** to implement any of these, but don't miss the fun if you have time!

- Alternative optimizers: you can try Adam, Adagrad, RMSprop, etc.
- Alternative activation functions such as leaky ReLU, parametric ReLU, ELU, or MaxOut.
- Model ensembles
- Data augmentation
- New Architectures
  - [ResNets](https://arxiv.org/abs/1512.03385) where the input from the previous layer is added to the output.
  - [DenseNets](https://arxiv.org/abs/1608.06993) where inputs into previous layers are concatenated together.
  - [This blog has an in-depth overview](https://chatbotslife.com/resnets-highwaynets-and-densenets-oh-my-9bb15918ee32)

### Have fun and happy training! 

In [21]:
################################################################################
# TODO:                                                                        #         
# Experiment with any architectures, optimizers, and hyperparameters.          #
# Achieve AT LEAST 70% accuracy on the *validation set* within 10 epochs.      #
#                                                                              #
# Note that you can use the check_accuracy function to evaluate on either      #
# the test set or the validation set, by passing either loader_test or         #
# loader_val as the second argument to check_accuracy. You should not touch    #
# the test set until you have finished your architecture and  hyperparameter   #
# tuning, and only run the test set once at the end to report a final value.   #
################################################################################
in_channel = 3
channel_1 = 32
channel_2 = 16
p = 0.2
## need to test/debug on dims
model = nn.Sequential(
    nn.Conv2d(in_channel, channel_1, 5, padding = 2),
    nn.BatchNorm2d(channel_1),
    nn.ReLU(),
    nn.Dropout(p),
    nn.Conv2d(channel_1, channel_2, 3, padding = 1),
    nn.BatchNorm2d(channel_2),
    nn.ReLU(),
    nn.Dropout(p),
    Flatten(),
    nn.Linear(channel_2 * 32 * 32, num_classes),
    nn.BatchNorm1d(num_classes, affine = False)
    )

optimizer = optim.SGD(model.parameters(), lr=learning_rate,
                     momentum=0.9, nesterov=True)

################################################################################
#                                 END OF YOUR CODE                             
################################################################################

train_part34(model, optimizer, epochs=3)

Iteration 0, loss = 2.8658
Checking accuracy on validation set
Got 146 / 1000 correct (14.60)

Iteration 100, loss = 1.6574
Checking accuracy on validation set
Got 448 / 1000 correct (44.80)

Iteration 200, loss = 1.5563
Checking accuracy on validation set
Got 491 / 1000 correct (49.10)

Iteration 300, loss = 1.5216
Checking accuracy on validation set
Got 525 / 1000 correct (52.50)

Iteration 400, loss = 1.4160
Checking accuracy on validation set
Got 522 / 1000 correct (52.20)

Iteration 500, loss = 1.3639
Checking accuracy on validation set
Got 539 / 1000 correct (53.90)

Iteration 600, loss = 1.4854
Checking accuracy on validation set
Got 526 / 1000 correct (52.60)

Iteration 700, loss = 1.4851
Checking accuracy on validation set
Got 549 / 1000 correct (54.90)

Iteration 0, loss = 1.3587
Checking accuracy on validation set
Got 543 / 1000 correct (54.30)

Iteration 100, loss = 1.3074
Checking accuracy on validation set
Got 563 / 1000 correct (56.30)

Iteration 200, loss = 1.3403
Check

## Describe what you did 

In the cell below you should write an explanation of what you did, any additional features that you implemented, and/or any graphs that you made in the process of training and evaluating your network.

TODO: Describe what you did

## Test set -- run this only once

Now that we've gotten a result we're happy with, we test our final model on the test set (which you should store in best_model). Think about how this compares to your validation set accuracy.

In [22]:
best_model = model
check_accuracy_part34(loader_test, best_model)

Checking accuracy on test set
Got 6419 / 10000 correct (64.19)
